---
# Porftolio Stock Strategy | Markotvitz Approach 
---

### Table of contents

1. [Introduction]()
2. [Model]()
3. [Data Wrangling]()<br>
   - Libraries
   - Data Collection
   - Setting Initial Dataframe
4. [Simulations]()
5. [Setting the Dataframe]()
6. [Visualizations]()
7. [Conclutions]()




#### 1. Introduction

Its evident that nowadays the economic outlook its getting worse and worse every day. The pandemic effects, thar war tensions and the sharp slowdown in gdp for the main economies has led to stagflation all over the world.
So, considering the increasing possibility of a near future recession, here its proposed a portfolio strategy using stocks that had outperformed in past recessions in order to develop an alternative option for those investors who still want to keep their positions in the stock market without changing to the fixed one in a recession scenery. The strategy was made based on the Markovitz portfolio theory model approach.

#### 2. Model

$$ E(R_{p}) = \sum\limits_{j=1}w_{j}E(R_{i}) $$


$$\small R_{p} = return \ of \ portfolio $$ 
$$\small E_{i} = return \ of \ asset \ i$$ 
$$\small W_{j} = weight \ of \ asset \ j$$ 

<br>

$$ \sigma^{2}_{p} = \sum\limits_{j}w^{2}_{j}\sigma^{2}_{i}+\sum\limits_{j}\sum\limits_{j!=i}w_{i}w_{j}\sigma_{i}\sigma_{j}\rho_{ij}$$

$$\small \sigma^{2}_{p} = portfolio \ variance $$ 
$$\small \rho_{ij} = correlation \ between \ asset \ ij$$ 




#### 3. Data Wrangling

**Libraries**

In [ ]:
import pandas_datareader.data as web #To collect data
import datetime as dt #To specify start and end dates
import pandas as pd #To read files or work with dataframes
import matplotlib.pyplot as plt #To visualize
import numpy as np #To get random weights
plt.style.use('fivethirtyeight')